[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/notebooks/basic_features_overview.ipynb)


TabNet 사용시 제 성능을 내지 못하는 이유를 찾기 위해 처음부터 훑어보기

In [2]:
%%capture
!pip install -U pytorch-tabnet lightning wget jsonargparse[signatures]>=4.18.0
!pip install -U \
    cloud-tpu-client==0.10 \
    https://storage.googleapis.com/tpu-pytorch/tmp/colab_tmp_whl/torch_xla-2.0.0.dev20230516+colab-cp310-cp310-linux_x86_64.whl
!pip install torch==2.0.0+cpu torchvision==0.15.1+cpu torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cpu
# !pip install -U \
#     cloud-tpu-client==0.10 \
#     https://storage.googleapis.com/tpu-pytorch/lsiyuan-experiment/wheel/torch_xla-2.1.0-cp310-cp310-linux_x86_64.whl

TabNet Tutorial 따라하기

In [3]:
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)

import scipy

import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline
import os
import torch

os.environ['CUDA_VISIBLE_DEVICES'] = f"1"
torch.__version__

'2.0.0+cpu'

In [4]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
dataset_name = 'census-income'
out = Path(os.getcwd()+'/data/'+dataset_name+'.csv')

out.parent.mkdir(parents=True, exist_ok=True)
if out.exists():
    print("File already exists.")
else:
    print("Downloading file...")
    wget.download(url, out.as_posix())

In [5]:
train = pd.read_csv(out)
print(len(train))
target = ' <=50K'
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("VV_likely")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)
# check that pipeline accepts strings
# train.loc[train[target]==0, target] = "wealthy"
# train.loc[train[target]==1, target] = "not_wealthy"

32560
39 73
 State-gov 9
 Bachelors 16
 13 16
 Never-married 7
 Adm-clerical 15
 Not-in-family 6
 White 5
 Male 2
 2174 119
 0 92
 40 94
 United-States 42
 <=50K 2
Set 3


In [6]:
unused_feat = ['Set']

features = [ col for col in train.columns if col not in unused_feat+[target]]
# 카테고리 index
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
# 카테고리별 class 개수
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

print("all features".rjust(25," "), len(features))
print("categoricals".rjust(25," "), cat_idxs)
print("categorical dimensions".rjust(25," ") , cat_dims)
grouped_features = [[0, 1, 2], [8, 9, 10]]

train.loc[train_indices, features+[target]].to_csv("train.csv", index=False)
train.loc[valid_indices, features+[target]].to_csv("valid.csv", index=False)
train.loc[test_indices, features+[target]].to_csv("test.csv", index=False)

             all features 14
             categoricals [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
   categorical dimensions [73, 9, 16, 16, 7, 15, 6, 5, 2, 119, 92, 94, 42]


In [ ]:
# steps, n_shared, n_independce 수정하지 않았음
tabnet_params = {
    "cat_idxs":cat_idxs,
    "cat_dims":cat_dims,
    "cat_emb_dim":2,
    "n_d": 16,
    "n_a": 16,
    "n_independent": 9,
    "n_shared": 4,
    "n_steps": 2,
    "gamma": 1.4690246460970766,
    "lambda_sparse": 0,
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params":dict(lr=2e-2),
    "scheduler_params":{
        "step_size":50, # how to use learning rate scheduler
        "gamma":0.9},
    "scheduler_fn":torch.optim.lr_scheduler.StepLR,
    "mask_type": 'entmax', # "sparsemax"
    "grouped_features" : grouped_features
   }

clf = TabNetClassifier(**tabnet_params)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [ ]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

놀랍게도 agumentations를 지원하고 있었다

In [ ]:
max_epochs = 50 if not os.getenv("CI", False) else 2

from pytorch_tabnet.augmentations import ClassificationSMOTE
aug = ClassificationSMOTE(p=0.2)

In [ ]:
# This illustrates the behaviour of the model's fit method using Compressed Sparse Row matrices
sparse_X_train = scipy.sparse.csr_matrix(X_train)  # Create a CSR matrix from X_train
sparse_X_valid = scipy.sparse.csr_matrix(X_valid)  # Create a CSR matrix from X_valid

# Fitting the model
clf.fit(
    X_train=sparse_X_train, y_train=y_train,
    eval_set=[(sparse_X_train, y_train), (sparse_X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False,
    augmentations=aug, #aug, None
)

epoch 0  | loss: 0.64829 | train_auc: 0.77059 | valid_auc: 0.76213 |  0:00:25s
epoch 1  | loss: 0.50207 | train_auc: 0.85442 | valid_auc: 0.85818 |  0:00:46s
epoch 2  | loss: 0.47382 | train_auc: 0.86842 | valid_auc: 0.87363 |  0:00:58s
epoch 3  | loss: 0.4549  | train_auc: 0.88284 | valid_auc: 0.88606 |  0:01:09s
epoch 4  | loss: 0.43946 | train_auc: 0.8888  | valid_auc: 0.89114 |  0:01:22s
epoch 5  | loss: 0.42655 | train_auc: 0.8944  | valid_auc: 0.89693 |  0:01:32s
epoch 6  | loss: 0.42313 | train_auc: 0.90136 | valid_auc: 0.90255 |  0:01:45s
epoch 7  | loss: 0.41862 | train_auc: 0.9059  | valid_auc: 0.90625 |  0:01:55s
epoch 8  | loss: 0.41363 | train_auc: 0.90829 | valid_auc: 0.90865 |  0:02:12s
epoch 9  | loss: 0.40659 | train_auc: 0.91225 | valid_auc: 0.91197 |  0:02:25s
epoch 10 | loss: 0.40391 | train_auc: 0.91637 | valid_auc: 0.91706 |  0:02:37s
epoch 11 | loss: 0.39921 | train_auc: 0.91785 | valid_auc: 0.91749 |  0:02:48s
epoch 12 | loss: 0.38973 | train_auc: 0.92165 | vali

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
# This illustrates the warm_start=False behaviour
save_history = []

# Fitting the model without starting from a warm start nor computing the feature importance
for _ in range(2):
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['auc'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False,
        augmentations=aug, #aug, None
        compute_importance=False
    )
    save_history.append(clf.history["valid_auc"])

assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

save_history = []  # Resetting the list to show that it also works when computing feature importance

# Fitting the model without starting from a warm start but with the computing of the feature importance activated
for _ in range(2):
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['auc'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False,
        augmentations=aug, #aug, None
        compute_importance=True # True by default so not needed
    )
    save_history.append(clf.history["valid_auc"])

assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

epoch 0  | loss: 0.64829 | train_auc: 0.77059 | valid_auc: 0.76213 |  0:00:03s
epoch 1  | loss: 0.50207 | train_auc: 0.85442 | valid_auc: 0.85818 |  0:00:08s
epoch 2  | loss: 0.47382 | train_auc: 0.86842 | valid_auc: 0.87363 |  0:00:14s
epoch 3  | loss: 0.4549  | train_auc: 0.88284 | valid_auc: 0.88606 |  0:00:20s
epoch 4  | loss: 0.43946 | train_auc: 0.8888  | valid_auc: 0.89114 |  0:00:24s
epoch 5  | loss: 0.42655 | train_auc: 0.8944  | valid_auc: 0.89693 |  0:00:27s
epoch 6  | loss: 0.42313 | train_auc: 0.90136 | valid_auc: 0.90255 |  0:00:33s
epoch 7  | loss: 0.41862 | train_auc: 0.9059  | valid_auc: 0.90625 |  0:00:39s
epoch 8  | loss: 0.41363 | train_auc: 0.90829 | valid_auc: 0.90865 |  0:00:43s
epoch 9  | loss: 0.40659 | train_auc: 0.91225 | valid_auc: 0.91197 |  0:00:47s
epoch 10 | loss: 0.40391 | train_auc: 0.91637 | valid_auc: 0.91706 |  0:00:51s
epoch 11 | loss: 0.39921 | train_auc: 0.91785 | valid_auc: 0.91749 |  0:00:58s
epoch 12 | loss: 0.38973 | train_auc: 0.92165 | vali

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.64829 | train_auc: 0.77059 | valid_auc: 0.76213 |  0:00:03s
epoch 1  | loss: 0.50207 | train_auc: 0.85442 | valid_auc: 0.85818 |  0:00:07s
epoch 2  | loss: 0.47382 | train_auc: 0.86842 | valid_auc: 0.87363 |  0:00:12s
epoch 3  | loss: 0.4549  | train_auc: 0.88284 | valid_auc: 0.88606 |  0:00:18s
epoch 4  | loss: 0.43946 | train_auc: 0.8888  | valid_auc: 0.89114 |  0:00:23s
epoch 5  | loss: 0.42655 | train_auc: 0.8944  | valid_auc: 0.89693 |  0:00:27s
epoch 6  | loss: 0.42313 | train_auc: 0.90136 | valid_auc: 0.90255 |  0:00:31s
epoch 7  | loss: 0.41862 | train_auc: 0.9059  | valid_auc: 0.90625 |  0:00:38s
epoch 8  | loss: 0.41363 | train_auc: 0.90829 | valid_auc: 0.90865 |  0:00:48s
epoch 9  | loss: 0.40659 | train_auc: 0.91225 | valid_auc: 0.91197 |  0:00:52s
epoch 10 | loss: 0.40391 | train_auc: 0.91637 | valid_auc: 0.91706 |  0:00:56s
epoch 11 | loss: 0.39921 | train_auc: 0.91785 | valid_auc: 0.91749 |  0:01:02s
epoch 12 | loss: 0.38973 | train_auc: 0.92165 | vali

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.64829 | train_auc: 0.77059 | valid_auc: 0.76213 |  0:00:03s
epoch 1  | loss: 0.50207 | train_auc: 0.85442 | valid_auc: 0.85818 |  0:00:07s
epoch 2  | loss: 0.47382 | train_auc: 0.86842 | valid_auc: 0.87363 |  0:00:13s
epoch 3  | loss: 0.4549  | train_auc: 0.88284 | valid_auc: 0.88606 |  0:00:19s
epoch 4  | loss: 0.43946 | train_auc: 0.8888  | valid_auc: 0.89114 |  0:00:23s
epoch 5  | loss: 0.42655 | train_auc: 0.8944  | valid_auc: 0.89693 |  0:00:27s
epoch 6  | loss: 0.42313 | train_auc: 0.90136 | valid_auc: 0.90255 |  0:00:31s
epoch 7  | loss: 0.41862 | train_auc: 0.9059  | valid_auc: 0.90625 |  0:00:37s
epoch 8  | loss: 0.41363 | train_auc: 0.90829 | valid_auc: 0.90865 |  0:00:43s
epoch 9  | loss: 0.40659 | train_auc: 0.91225 | valid_auc: 0.91197 |  0:00:47s
epoch 10 | loss: 0.40391 | train_auc: 0.91637 | valid_auc: 0.91706 |  0:00:51s
epoch 11 | loss: 0.39921 | train_auc: 0.91785 | valid_auc: 0.91749 |  0:00:56s
epoch 12 | loss: 0.38973 | train_auc: 0.92165 | vali

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.64829 | train_auc: 0.77059 | valid_auc: 0.76213 |  0:00:04s
epoch 1  | loss: 0.50207 | train_auc: 0.85442 | valid_auc: 0.85818 |  0:00:08s
epoch 2  | loss: 0.47382 | train_auc: 0.86842 | valid_auc: 0.87363 |  0:00:12s
epoch 3  | loss: 0.4549  | train_auc: 0.88284 | valid_auc: 0.88606 |  0:00:18s
epoch 4  | loss: 0.43946 | train_auc: 0.8888  | valid_auc: 0.89114 |  0:00:24s
epoch 5  | loss: 0.42655 | train_auc: 0.8944  | valid_auc: 0.89693 |  0:00:27s
epoch 6  | loss: 0.42313 | train_auc: 0.90136 | valid_auc: 0.90255 |  0:00:31s
epoch 7  | loss: 0.41862 | train_auc: 0.9059  | valid_auc: 0.90625 |  0:00:36s
epoch 8  | loss: 0.41363 | train_auc: 0.90829 | valid_auc: 0.90865 |  0:00:42s
epoch 9  | loss: 0.40659 | train_auc: 0.91225 | valid_auc: 0.91197 |  0:00:47s
epoch 10 | loss: 0.40391 | train_auc: 0.91637 | valid_auc: 0.91706 |  0:00:51s
epoch 11 | loss: 0.39921 | train_auc: 0.91785 | valid_auc: 0.91749 |  0:00:55s
epoch 12 | loss: 0.38973 | train_auc: 0.92165 | vali

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
# plot losses
plt.plot(clf.history['loss'])

In [ ]:
# plot auc
plt.plot(clf.history['train_auc'])
plt.plot(clf.history['valid_auc'])

In [ ]:
# plot learning rates
plt.plot(clf.history['lr'])

In [ ]:
preds = clf.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)


preds_valid = clf.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=preds_valid[:,1], y_true=y_valid)

print(f"BEST VALID SCORE FOR {dataset_name} : {clf.best_cost}")
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")
# check that best weights are used
assert np.isclose(valid_auc, np.max(clf.history['valid_auc']), atol=1e-6)
clf.predict(X_test)


In [ ]:
# save tabnet model
saving_path_name = "./tabnet_model_test_1"
saved_filepath = clf.save_model(saving_path_name)

# define new model with basic parameters and load state dict weights
loaded_clf = TabNetClassifier()
loaded_clf.load_model(saved_filepath)

In [ ]:
loaded_preds = loaded_clf.predict_proba(X_test)
loaded_test_auc = roc_auc_score(y_score=loaded_preds[:,1], y_true=y_test)

print(f"FINAL TEST SCORE FOR {dataset_name} : {loaded_test_auc}")

assert(test_auc == loaded_test_auc)
loaded_clf.predict(X_test)


In [ ]:
# Global explainability : feat importance summing to 1
clf.feature_importances_

In [ ]:
explain_matrix, masks = clf.explain(X_test)
fig, axs = plt.subplots(1, 3, figsize=(20,20))

for i in range(3):
    axs[i].imshow(masks[i][:50])
    axs[i].set_title(f"mask {i}")
    axs[i].set_xticklabels(labels = features, rotation=45)

Lightning 으로 네트워크 학습하기

In [23]:
%%writefile tabnet.py
# @title <b><i>Training 파일 저장<i/></b>
import argparse
import logging
import os
import gc
import sys
import time
import pickle
from typing import Any, Dict, Optional, Type
# import dgl
import numpy as np
import pandas as pd
# import deepchem as dc

import torch
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torchmetrics import F1Score, AUROC

from torch.optim import SGD, Adam, NAdam, RAdam, SparseAdam, LBFGS, RMSprop
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CyclicLR
# from torchmetrics.text.bleu import BLEUScore
# from dgl.dataloading import GraphDataLoader
from pytorch_tabnet.tab_network import TabNetPretraining, TabNet
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.utils import create_group_matrix, create_explain_matrix
from lightning.fabric.utilities import apply_func
from torchsummary import summary

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import (
    TQDMProgressBar, EarlyStopping, LearningRateMonitor,
    ModelCheckpoint, StochasticWeightAveraging, DeviceStatsMonitor)
from lightning.pytorch.plugins.io import XLACheckpointIO
from lightning.pytorch.cli import LightningCLI, ArgsType
from lightning_utilities.core.imports import module_available

sys.setrecursionlimit(10**7)
sys.set_int_max_str_digits(0)
isxla = module_available("torch_xla")
print(f'Can Use Torch_XLA? {isxla}')
if isxla:
    import torch_xla
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.xla_multiprocessing as xmp
    import torch_xla.utils.serialization as xser

logger = logging.getLogger()
logger.setLevel(logging.WARNING)


class ArgsCLI(LightningCLI):
    def add_arguments_to_parser(self, parser):
        parser.add_argument('--train_file',
                            type=str,
                            default='dev_t.csv',
                            help='train file')

        parser.add_argument('--test_file',
                            type=str,
                            default='dev_v.csv',
                            help='test file')


        parser.add_argument('--accumulate_grad_batches',
                            type=int,
                            default=1,
                            help='accumulate_grad_batches')

        parser.add_argument('--num_workers',
                            type=int,
                            default=2,
                            help='num of worker for dataloader')

        parser.add_argument('--batch_size',
                            type=int,
                            default=32,
                            help='batch size for training (default: 96)')

        parser.add_argument('--lr',
                            type=float,
                            default=5e-7,
                            help='The initial learning rate')

        parser.add_argument('--warmup_ratio',
                            type=float,
                            default=0.1,
                            help='warmup ratio')


        parser.add_argument('--checkpoint_path',
                            type=str,
                            help='checkpoint path')

        parser.add_argument('--default_root_dir',
                            type=str,
                            help='default_root_dir')


class CensusDataset(Dataset):
    def __init__(self,
                 filepath: str,) -> None:
        super().__init__()
        self.data = pd.read_csv(filepath)

    def __len__(self):
        return len(self.data)

    def update_attributes(self, features, idxs, dims):
        self.features=features
        self.cat_ids = idxs
        self.cat_dims = dims

    def make_input_id_mask(self,
                           tokens):
        input_id = self.tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        if len(input_id) < self.max_seq_len:
            while len(input_id) < self.max_seq_len:
                input_id += [self.tokenizer.pad_token_id]
                attention_mask += [0]
        else:
            input_id = input_id[:self.max_seq_len - 1] + [self.tokenizer.eos_token_id]
            attention_mask = attention_mask[:self.max_seq_len]
        return input_id, attention_mask

    def masking(self,
                input,
                mask):
        input = torch.tensor([input])
        rand = torch.rand(input.shape)
        mask_arr = (rand < 0.15) * (input != self.tokenizer.bos_token_id) * (input != self.tokenizer.eos_token_id)
        mask_ids = torch.flatten((mask_arr[0]).nonzero()).tolist()
        input[0, mask_ids] = mask
        del rand, mask_arr
        return input.tolist()[0]

    def _labeling(self,
                  label):
        tokens = [self.tokenizer.bos_token]+self.tokenizer.tokenize(label)+[self.tokenizer.eos_token]
        label_ids = self.tokenizer.convert_tokens_to_ids(tokens[1:])
        if len(label_ids) < self.max_seq_len:
            while len(label_ids)<self.max_seq_len:
                label_ids+=[-100]
        else:
            label_ids = label_ids[:self.max_seq_len-1] + [self.tokenizer.eos_token_id]
        del tokens
        return label_ids

    def _scale_transform(self, values):
        return self.scaler.transform(values.reshape(1,-1))

    def __getitem__(self,
                    index:int):
        assert self.features
        record = self.data.iloc[index]
        dod  = record[' <=50K']
        tabs = record[self.features].values
        return {
            'tabnet_input': tabs.astype('float32'),
            'labels': np.array(dod, dtype=np.float32)
        }


class CensusDataModule(pl.core.LightningDataModule):
    def __init__(self,
                 train_file: str,
                 valid_file: str,
                 test_file: str,
                 batch_size: int = 32,
                 num_workers: int = 8):
        super().__init__()
        self.batch_size = batch_size # batch_size
        self.train_file_path = train_file
        self.valid_file_path = valid_file
        self.test_file_path = test_file
        self.num_workers = num_workers
        self.prepare_data_pre_node = True
    # OPTIONAL, called for every GPU/machine (assigning state is OK)

    def setup(self,
              stage):
        # split dataset
        self.train = CensusDataset(self.train_file_path)
        self.valid = CensusDataset(self.valid_file_path)
        self.test = CensusDataset(self.test_file_path)
        features, idxs, dims = self.get_features()
        self.train.update_attributes(features, idxs, dims)
        self.valid.update_attributes(features, idxs, dims)
        self.test.update_attributes(features, idxs, dims)

    def get_features(self):
        df = pd.concat([self.train.data, self.valid.data, self.test.data])
        target = ' <=50K'
        nunique = df.nunique()
        types = df.dtypes
        categorical_columns = []
        categorical_dims =  {}
        for col in df.columns:
            if types[col] == 'object' or nunique[col] < 200:
                l_enc = LabelEncoder()
                df[col] = df[col].fillna("VV_likely")
                df[col] = l_enc.fit_transform(df[col].values)
                categorical_columns.append(col)
                categorical_dims[col] = len(l_enc.classes_)
            else:
                df.fillna(df.median(), inplace=True)
        unused_feat = ['Set']
        features = [ col for col in df.columns if col not in unused_feat+[target]]
        cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
        cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]
        return features, cat_idxs, cat_dims

    def train_dataloader(self):
        return DataLoader(self.train,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          pin_memory=True,
                          shuffle=True,
                          drop_last=True)

    def val_dataloader(self):
        return DataLoader(self.valid,
                          batch_size=self.batch_size,
                          num_workers= self.num_workers,
                          pin_memory=True,
                          shuffle=False,
                          drop_last=True)

    def test_dataloader(self):
        return DataLoader(self.test,
                          batch_size=self.batch_size,
                          num_workers= self.num_workers,
                          pin_memory=True,
                          shuffle=False,
                          drop_last=True)



class Base(pl.core.LightningModule):
    def __init__(self,
                 lr: float,
                 batch_size: int,
                 warmup: float = 0.1,
                 **kwargs) -> None:
        super().__init__()
        self.lr = lr
        self.warmup_ratio = warmup
        self.batch_size = batch_size

    def decay_params(self, named_params):
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight', 'BatchNorm1d.bias', 'BatchNorm1d.weight']
        return [
            {'params': [p for n, p in named_params if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in named_params if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

    def configure_optimizers(self):
        # Prepare optimizer

        # optimizer_grouped_parameters = self.network.net.parameters()
        optimizer_grouped_parameters = self.decay_params(self.network.net.named_parameters())

        optimizer = Adam(params=self.parameters(),
                         lr=self.lr,)
        # optimizer = RMSprop(params=optimizer_grouped_parameters,
        #                     momentum=0.9,
        #                     lr=self.lr)

        # warm up lr
        num_nodes = self.trainer.strategy.num_nodes if not "Single" in str(type(self.trainer.strategy)) else 1
        devices = len(self.trainer.strategy.parallel_devices) if "parallel_devices" in self.trainer.strategy.__dict__.keys() else 1
        num_devices = int(devices) * int(num_nodes)
        data_len = len(self.trainer.fit_loop._data_source.instance.train)
       # data_len = len(self.trainer._data_connector._train_dataloader_source.dataloader().dataset)
        logging.info(f'number of devices {num_devices}, data length {data_len}')
        first = (data_len / (self.batch_size * num_devices))
        num_train_steps = int(first * self.trainer.max_epochs)
        logging.info(f'num_train_steps : {num_train_steps}')

        num_warmup_steps = int(num_train_steps * 0.01)
        logging.info(f'num_warmup_steps : {num_warmup_steps}')

        scheduler = CyclicLR(
                 optimizer=optimizer,
                 base_lr=5e-5,
                 max_lr=self.lr,
                 cycle_momentum=False,
                 step_size_up=num_warmup_steps,)

        self.lr_scheduler = {
            "scheduler": scheduler,
            "name": "lr_log",
            "monitor": "loss",
            "interval": "step",
            "frequency": 1}
        # return optimizer
        return [optimizer], [self.lr_scheduler]

   # def backward(self, loss):
       # with torch.autograd.set_detect_anomaly(True):
       # loss.to(self.device)
       # loss.requires_grad_()
       # loss.backward()



class Tabnet(torch.nn.Module):
    def __init__(self,
                 input_dim: int,
                 output_dim: int,
                 batch_size: int,
                 cat_idx:list,
                 cat_dim:list,
                 device: str = "cpu"):
        super(Tabnet, self).__init__()
        self.cat_idx = cat_idx
        self.cat_dim = cat_dim
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.batch_size = batch_size
        self.net = TabNet(
            cat_idxs=cat_idx,
            cat_dims=cat_dim,
            cat_emb_dim=[3]*len(cat_idx),
            input_dim=input_dim,
            output_dim=output_dim,
            n_a=2, # Dimension of the prediction layer
            n_d=2, # Dimension of the attention layer
            n_steps=3, # Output까지의 Attentive-Feature transformer layer
            n_independent=1, # Decision step dependent 단계 GLU layer
            n_shared=1, # Shared across decision steps 단계 GLU layer
            epsilon=1e-5,
            gamma=1.3,
            mask_type="sparsemax", # sparsemax / entmax
            momentum=0.01,
            group_attention_matrix=torch.eye(input_dim, device=device))
        self.reducing_matrix = create_explain_matrix(
                    self.net.input_dim,
                    self.net.cat_emb_dim,
                    self.net.cat_idxs,
                    self.net.post_embed_dim,
                )

    def _re_init(self, device, state_dict=None):
       # if "xla" in device:
        del self.net, self.reducing_matrix
        self.__init__(
                input_dim=self.input_dim,
                output_dim=self.output_dim,
                batch_size=self.batch_size,
                cat_idx=self.cat_idx,
                cat_dim=self.cat_dim,
                device=device)
        if state_dict:
            self.load_state_dict(state_dict)
        self.to(device)

    def forward(self, x):
        output, loss = self.net(x)
        return output, loss


class LightningTabNet(Base):
    def __init__(self,
                 lr: float,
                 batch_size: int,
                 warmup: float = 0.1,
                 **kwargs):
        super().__init__(lr, warmup, batch_size, **kwargs)
        model_dim = 2 # @param {type:"integer"}
        cat_input_dim = 14 # @param {type:"integer"}
        self.network = Tabnet(
            input_dim=cat_input_dim,
            output_dim=model_dim,
            batch_size=batch_size,
            cat_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
            cat_dim =  [73, 9, 16, 16, 7, 15, 6, 5, 2, 119, 92, 94, 42])
        self.f1 = F1Score(task="binary", num_classes=2)
        self.roc = AUROC(task="binary", num_classes=2)
       # self.concat = torch.nn.Bilinear(model_dim, model_dim, model_dim,)

    def _loss(self, yhat, y, M_loss):
        pred = yhat[:, 1].gt(0.5).float().to(self.device)
       # pred = F.log_softmax(yhat, dim=1).to(self.device)
        acc = torch.sum(torch.eq(pred, y))/ pred.shape[0]
        f1 = self.f1(pred, y)
        roc = self.roc(pred, y)
        clf_loss = F.cross_entropy(yhat, y, )
        clf_loss = clf_loss - (1.3 * M_loss)
        clf_loss.requires_grad_()
        clf_loss.to(self.device)
        return clf_loss, acc, f1, roc

    def initialize(self, m):
        for layer in m.modules():
            if isinstance(layer, torch.nn.Linear) or isinstance(layer, torch.nn.Bilinear):
                torch.nn.init.xavier_uniform_(layer.weight, gain=1.0)

   # def teardown(self,
   #              trainer: pl.Trainer,
   #              pl_module: pl.LightningModule,
   #              stage: str) -> None:
   #     print(f"{stage} Done")

    def on_fit_start(self,) -> None:
        self.network._re_init(self.device) # Multiprocessing 하는 경우 device에 올라가는 문제
       # self.lm.init_weights()
       # self.initialize(self.tabnet.net)
       # print("Training with Lightning Module\n")

    def on_predict_start(self,) -> None:
        state_dict = self.network.state_dict()
       # self.tabnet._re_init(self.device, state_dict)

   # def on_fit_end(self,) -> None:
   #     if xm.is_master_ordinal():
   #         print("\nTrain Done\n")
   #         # save_hf_repo(self.model.cpu(), self.tokenizer)

    def forward(self,
                inputs):
        output, M_loss = self.network(inputs["tabnet_input"])
       # output = torch.sum(torch.nan_to_num(torch.stack(output)), dim=0)
        return output, M_loss

   # def on_train_epoch_start(self):
   #     self.lm.train()
   #     self.tabnet.train()

    def training_step(self,
                      batch,
                      batch_idx: int):
        outs, M_loss = self(batch)
        loss, dod_acc, f1, roc = self._loss(
            yhat=outs,
            y=batch["labels"],
            M_loss=M_loss)

        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
        self.log("train_dod_acc", dod_acc, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
        self.log("train_f1", f1, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
        self.log("train_roc", roc, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
        return loss

    def validation_step(self,
                        batch,
                        batch_idx: int):
        outs, M_loss = self(batch)
        loss, dod_acc, f1, roc = self._loss(
            yhat=outs,
            y=batch["labels"],
            M_loss=M_loss)
        self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True)
        self.log("val_dod_acc", dod_acc, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True)
        self.log("val_f1", f1, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
        self.log("val_roc", roc, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)


    # def on_training_batch_end(self,
    #                           outputs,
    #                           batch,
    #                           batch_idx) -> None:
    #     print("Train Batch End")

    # def on_training_epoch_end(self,) -> None:
    #     print("Train Epoch End")

    # def validation_batch_end(self,
    #                          outputs,
    #                          batch,
    #                          batch_idx,
    #                          dataloader_idx=0) -> None:
    #     print("Valid Batch End")

    # def validation_epoch_end(self,) -> None:
    #     print("Valid Epoch End")

    # def on_save_checkpoint(self,
    #                        trainer: pl.Trainer,
    #                        pl_module: pl.LightningModule=None,
    #                        checkpoint: Dict[str,Any]=None) -> None:
    #     checkpoint["lighitning_logs/checkpoint"]=self
    #     if xm.is_master_ordinal():
    #         print(f"Trainer save checkpoint")
    #         save_hf_repo(self.model.cpu(), self.model.tokenizer)
    #         print("\nCheckpoint saved\n")


def cli_main() -> None: # args:ArgsType=None
    callbacks = [
        # ModelCheckpoint(monitor="val_loss",
        #                dirpath="logs/",
        #                filename="best-checkpoint",
        #                verbose=True,
        #                save_last=True,
        #                mode="min",
        #                save_top_k=1),
        # TensorBoardLogger(save_dir="lightning_logs/"),
        # StochasticWeightAveraging(swa_lrs=1e-2), # 23/04/24 기준 lightning에서 오류 있는 callback
        # DeviceStatsMonitor(),
        # EarlyStopping(monitor="val_loss",
        #               mode="min",
        #               stopping_threshold=1e-4,
        #               min_delta=0.00,
        #               patience=1,
        #               divergence_threshold=9.0),
        LearningRateMonitor(logging_interval="step"),
        TQDMProgressBar(refresh_rate=3)]

    cli = ArgsCLI(model_class=LightningTabNet,
                  datamodule_class=CensusDataModule,
                  save_config_kwargs={"overwrite": True},
                  trainer_defaults={
                    # "plugins": [XLACheckpointIO(),],
                      "callbacks": callbacks,
                      "reload_dataloaders_every_n_epochs": 1,
                      "detect_anomaly": True,
                      },)

if __name__ == "__main__":
    pl.cli_lightning_logo()
    cli_main()

   # save_path = "/content/drive/MyDrive/data_kevin/ChemGPT_finetuned.pth"
   # torch.save(model.model.state_dict(),save_path)
    print("\nMain Done...\n")


Overwriting tabnet.py


In [24]:
%%writefile trainer.sh
TRAIN_FILE="train.csv" #@param ['"train.csv"'] {type:"raw"}
VAL_FILE="valid.csv" #@param ['"test.csv"'] {type:"raw"}
TEST_FILE="test.csv" #@param ['"test.csv"'] {type:"raw"}
PERCISION=32 #@param ["bf16", "16", "32"] {type:"raw", allow-input: true}
LEARNING_RATE=2e-02 #@param {type:"raw"}
GRADIENT_CLIP=1.0 #@param {type:"raw"}
ACCUMULATE_GRAD_BATCHES=1 #@param {type:"raw"}
BATCH_SIZE=64 #@param ["1", "2", "4", "8", "16", "32", "64", "128", "512", "4096", "32768", "262144", "2097152", "16777216", "134217728", "1073741824", "8589934592", "68719476736"] {type:"raw", allow-input: true}
MAXEPOCHS=100 #@param {type:"raw"}
STRATEGY=xla #@param ["auto", "bagua", "colossalai", "collaborative", "ddp", "ddp2", "ddp_find_unused_parameters_false", "ddp_fully_sharded", "ddp_sharded", "ddp_sharded_find_unused_parameters_false", "ddp_sharded_spawn", "ddp_sharded_spawn_find_unused_parameters_false", "ddp_spawn", "ddp_spawn_find_unused_parameters_false", "ddp_fork" ,"deepspeed", "deepspeed_stage_1", "deepspeed_stage_2", "deepspeed_stage_2_offload", "deepspeed_stage_3", "deepspeed_stage_3_offload", "deepspeed_stage_3_offload_nvme", "dp", "fsdp", "fsdp_native", "horovod", "hpu_parallel", "hpu_single", "ipu_strategy", "single_device", "single_tpu", "tpu_spawn", "tpu_spawn_debug", "xla", "xla_debug"]{type:"raw", allow-input: false}
NUMWORKERS=0 #@param {type:"raw"}
WARMUP=0.1 #@param {type:"raw"}
PROFILE=xla #@param ["simple", "advanced", "pytorch", "xla"]  {type:"raw", allow-input: false}
FAST_DEV_RUN=false #@param ["true", "false"] {type:"raw", allow-input: false}
SYNC_BATCHNORM=true #@param ["true", "false"] {type:"raw", allow-input: false}
ACCELERATOR=tpu #@param ["gpu", "tpu"] {type:"raw", allow-input: true}
CORES=8 #@param {type:"raw"}

python tabnet.py fit\
    --trainer.accumulate_grad_batches $ACCUMULATE_GRAD_BATCHES\
    --trainer.gradient_clip_val $GRADIENT_CLIP\
    --trainer.max_epochs $MAXEPOCHS\
    --trainer.precision $PERCISION\
    --trainer.accelerator $ACCELERATOR\
    --trainer.devices $CORES\
    --trainer.use_distributed_sampler false\
    --trainer.sync_batchnorm $SYNC_BATCHNORM\
    --trainer.profiler $PROFILE\
    --trainer.strategy $STRATEGY\
    --trainer.num_sanity_val_steps 3\
    --trainer.log_every_n_steps 2\
    --trainer.fast_dev_run $FAST_DEV_RUN\
    --model.lr $LEARNING_RATE\
    --model.batch_size $BATCH_SIZE\
    --data.num_workers $NUMWORKERS\
    --data.batch_size $BATCH_SIZE\
    --model.warmup $WARMUP\
    --data.train_file $TRAIN_FILE\
    --data.valid_file $VAL_FILE\
    --data.test_file $TEST_FILE

exit 0

Overwriting trainer.sh


In [26]:
!sh trainer.sh

Can Use Torch_XLA? True


                    ####
                ###########
             ####################
         ############################
    #####################################
##############################################
#########################  ###################
#######################    ###################
####################      ####################
##################       #####################
################        ######################
#####################        #################
######################     ###################
#####################    #####################
####################   #######################
###################  #########################
##############################################
    #####################################
         ############################
             ####################
                  ##########
                     ####


/usr/local/lib/python3.10/dist-packages/lightning/fabric/utilities/seed.